<div class="alert alert-block alert-info">
Направил на вторую проверку не ту тетрадку, прошу прощение
</div>


# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
!pip install scikit-learn==1.1.3

## Подготовка данных

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import math
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor
from sklearn.pipeline import make_pipeline
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("/datasets/autos.csv")
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

None

In [5]:
data.duplicated().sum()

4

In [6]:
data[data.duplicated(keep=False)]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
41529,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15
88087,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
90964,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
171088,2016-03-08 18:42:48,1799,coupe,1999,auto,193,clk,20000,7,petrol,mercedes_benz,no,2016-03-08 00:00:00,0,89518,2016-03-09 09:46:57
187735,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
231258,2016-03-28 00:56:10,1000,small,2002,manual,83,other,150000,1,petrol,suzuki,no,2016-03-28 00:00:00,0,66589,2016-03-28 08:46:21
258109,2016-04-03 09:01:15,4699,coupe,2003,auto,218,clk,125000,6,petrol,mercedes_benz,yes,2016-04-03 00:00:00,0,75196,2016-04-07 09:44:54
325651,2016-03-18 18:46:15,1999,wagon,2001,manual,131,passat,150000,7,gasoline,volkswagen,no,2016-03-18 00:00:00,0,36391,2016-03-18 18:46:15


In [7]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [8]:
print("Колличество объявлений с нулевой ценой:",len(data.loc[data['Price'] == 0]))

Колличество объявлений с нулевой ценой: 10772


In [9]:
data = data.loc[data['Price'] != 0]
print("Колличество объявлений с нулевой ценой:",len(data.loc[data['Price'] == 0]))

Колличество объявлений с нулевой ценой: 0


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Удалили некорректные значения целевого признака, заменять их увы не получилось бы из-за того, что признак целевой
</div>

In [10]:
print("Колличество объявлений с пропущенной моделью:", len(data.loc[data['Model'].isna()]))

Колличество объявлений с пропущенной моделью: 17521


In [11]:
data = data.loc[~data['Model'].isna()]
print("Колличество объявлений с пропущенной моделью:", len(data.loc[data['Model'].isna()]))

Колличество объявлений с пропущенной моделью: 0


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Эти данные так же не было возможности восстановить
</div>

In [12]:
print("Колличество объявлений с незаполненным типом кузова:", len(data.loc[data['VehicleType'].isna()]))

Колличество объявлений с незаполненным типом кузова: 28166


In [13]:
data['VehicleType'] = data['VehicleType'].fillna('unknown')
print("Колличество объявлений с незаполненным типом кузова:", len(data.loc[data['VehicleType'].isna()]))

Колличество объявлений с незаполненным типом кузова: 0


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Данных слишком много, поэтому заменили пропуски на неизвестно, чтобы это не так сильно отразилось на предсказание
</div>

In [14]:
print("Колличество объявлений с некорректной годом:",len(data.loc[(data['RegistrationYear'] > 2021) | (data['RegistrationYear'] < 1769)]))

Колличество объявлений с некорректной годом: 83


In [15]:
data = data.loc[(data['RegistrationYear'] <= 2021) & (data['RegistrationYear'] >= 1900)]
print("Колличество объявлений с некорректной годом:",len(data.loc[(data['RegistrationYear'] > 2021) | (data['RegistrationYear'] < 1769)]))

Колличество объявлений с некорректной годом: 0


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Данных мало удаляем
</div>

In [16]:
print("Колличество объявлений с незаполненным типом коробки:", len(data.loc[data['Gearbox'].isna()]))

Колличество объявлений с незаполненным типом коробки: 14205


In [17]:
data['Gearbox'] = data['Gearbox'].fillna(data
                                         .groupby('Model')['Gearbox']
                                         .transform(lambda x: x.value_counts().idxmax())
                                        )

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Заполнили данные самым популярным вариантом
</div>

In [18]:
print("Колличество объявлений с некорректной мощностью:",len(data.loc[(data['Power'] > 1000) | (data['Power'] <= 0)]))

Колличество объявлений с некорректной мощностью: 31519


In [19]:
data.loc[(data['Power'] > 1000) | (data['Power'] <= 0), 'Power'] = None
data['Power'] = data['Power'].fillna(data.groupby('Model')['Power'].transform('median'))
data = data.loc[~data['Power'].isna()]
data['Power'] = data['Power'].astype('int64')

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Некорректные данные мощности заменили на медиану
</div>

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Нет, потому что сервис по идее продает машины, которые на ходу и с пробегом, а отсутствие двигателя протеворечит этому 
</div>

In [20]:
data['FuelType'] = data['FuelType'].fillna(data.groupby('Model')['FuelType'].transform(lambda x: x.value_counts().idxmax()))

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Меняем значение на средне по моделям, так как это наиболее логично
</div>

In [21]:
data["Repaired"].fillna("no", inplace=True)

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Заменили пропуски на нет, так как пропуск скорее всего значит, что машина не была повреждена
</div>

In [22]:
print("Минимальная дата создания объявления:", min(data['DateCreated']))

Минимальная дата создания объявления: 2014-03-10 00:00:00


In [23]:
base_date = pd.Timestamp(min(data['DateCreated']))
data['DateCreated'] = data['DateCreated'].map(lambda date : (pd.Timestamp(date) - base_date).days)

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Преобразовали пропуски в самый раний день создания
</div>

In [24]:
data = data.drop_duplicates()

In [25]:
data = data.drop(['NumberOfPictures', 'PostalCode', 'DateCrawled', 'LastSeen'], axis=1)


In [26]:
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325984 entries, 0 to 325983
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              325984 non-null  int64 
 1   VehicleType        325984 non-null  object
 2   RegistrationYear   325984 non-null  int64 
 3   Gearbox            325984 non-null  object
 4   Power              325984 non-null  int64 
 5   Model              325984 non-null  object
 6   Kilometer          325984 non-null  int64 
 7   RegistrationMonth  325984 non-null  int64 
 8   FuelType           325984 non-null  object
 9   Brand              325984 non-null  object
 10  Repaired           325984 non-null  object
 11  DateCreated        325984 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 29.8+ MB


In [27]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,DateCreated
count,325984.000000,325984.000000,325984.000000,325984.000000,325984.000000,325984.000000
mean,4623.348821,2003.268133,119.117303,128597.078384,5.853956,741.798055
std,4530.471089,7.061796,53.172571,36963.803773,3.665991,9.399236
min,1.000000,1910.000000,1.000000,5000.000000,0.000000,0.000000
25%,1200.000000,1999.000000,75.000000,125000.000000,3.000000,734.000000
50%,2950.000000,2003.000000,110.000000,150000.000000,6.000000,742.000000
75%,6649.250000,2008.000000,145.000000,150000.000000,9.000000,750.000000
max,20000.000000,2019.000000,1000.000000,150000.000000,12.000000,759.000000


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Удалили дубликаты, ненужные признаки и проверили описание данных
</div>

## Обучение моделей

In [28]:
target = data['Price']
features = data.drop('Price', axis=1)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=42)

In [29]:
encoder  = OneHotEncoder(drop = 'first', sparse=False,handle_unknown='ignore')
cat_col = ['VehicleType', 'Gearbox', 'FuelType', 'Repaired','Model','Brand']

X_train_ohe = encoder.fit_transform(features_train[cat_col]) 
encoder_col_names = encoder.get_feature_names()
df_train_ohe = pd.DataFrame(X_train_ohe, columns=encoder_col_names)

X_test_ohe = encoder.transform(features_test[cat_col]) 
df_test_ohe = pd.DataFrame(X_test_ohe, columns=encoder_col_names)

In [30]:
features_train.reset_index(drop = True, inplace= True )
features_test.reset_index(drop = True, inplace= True )

features_train_final = features_train.join(df_train_ohe)
features_test_final = features_test.join(df_test_ohe)

features_train_final.drop(cat_col, axis= 1 , inplace= True)
features_test_final.drop(cat_col, axis= 1 , inplace= True)

In [31]:
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
features_train_lgb = features_train.copy()
features_train_lgb[cat_col] = ord_enc.fit_transform(features_train_lgb[cat_col]).astype(int)

features_test_lgb = features_test.copy()
features_test_lgb[cat_col] = ord_enc.transform(features_test_lgb[cat_col]).astype(int)

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Закодировали категориальные признаки
</div>

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
разбили данные на выборки
</div>

In [32]:
numeric = ['RegistrationYear', 'Power',  'Kilometer', 'RegistrationMonth',  'DateCreated']

In [33]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_test[numeric] = scaler.transform(features_test[numeric]) 
print(features_train.head())

  VehicleType  RegistrationYear Gearbox     Power     Model  Kilometer  \
0         suv         -0.322266    auto  0.821388     other   0.579154   
1       coupe         -0.605260  manual  1.327445       v70   0.579154   
2       coupe         -0.180770    auto  1.852246       clk   0.579154   
3       coupe         -0.888254  manual -0.228212  mx_reihe  -0.097122   
4         suv          0.526715    auto  1.964703  m_klasse   0.579154   

   RegistrationMonth  FuelType          Brand Repaired  DateCreated  
0           0.585722  gasoline         toyota       no     0.342208  
1          -1.596919    petrol          volvo       no     1.405102  
2           0.040062    petrol  mercedes_benz       no    -1.039553  
3          -0.778429    petrol          mazda      yes    -0.826974  
4           1.677043  gasoline  mercedes_benz       no     0.235919  


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
стандартизировали числовые данные
</div>

## Анализ моделей

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Ridge
</div>

In [33]:
rmse_score_diff_methods = pd.DataFrame()

In [34]:
model_ridge = Ridge(random_state=42)

pipeline_ridge = make_pipeline(model_ridge)

params_grid_ridge = {'ridge__alpha': np.arange(0.00, 0.25, 0.01)}

grid_ridge = GridSearchCV(
    pipeline_ridge, 
    param_grid=params_grid_ridge, 
    scoring='neg_root_mean_squared_error', 
    n_jobs=-1,
    verbose=1)

In [35]:
grid_ridge.fit(features_train_final, target_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(estimator=Pipeline(steps=[('ridge', Ridge(random_state=42))]),
             n_jobs=-1,
             param_grid={'ridge__alpha': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24])},
             scoring='neg_root_mean_squared_error', verbose=1)

In [36]:
best_model_ridge = grid_ridge.best_estimator_
best_parametrs_ridge = grid_ridge.best_params_
best_result_ridge = grid_ridge.best_score_ * -1

In [37]:
print(f'Лучший результат RMSE модели: {best_result_ridge}') # лучший результат RMSE модели
print(f'Лучшие параметры модели: {best_parametrs_ridge}') # лучшие параметры модели

Лучший результат RMSE модели: 2641.9277576017103
Лучшие параметры модели: {'ridge__alpha': 0.24}


In [38]:
%%time

predicted_test = best_model_ridge.predict(features_train_final)

CPU times: user 298 ms, sys: 150 ms, total: 449 ms
Wall time: 387 ms


In [39]:
rmse_score_diff_methods.loc['Ridge', 'RMSE'] = best_result_ridge
rmse_score_diff_methods.loc['Ridge', 'time_fit at s'] = grid_ridge.cv_results_['mean_fit_time'].mean()
rmse_score_diff_methods.loc['Ridge', 'time_predict'] = '119  ms'

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
DecisionTreeRegressor
</div>

In [40]:
model_tree = DecisionTreeRegressor(random_state=42)

pipline_tree = make_pipeline(model_tree)

params_grid_tree = {'decisiontreeregressor__max_depth': range(2, 15)}

grid_tree = GridSearchCV(
    pipline_tree, 
    param_grid=params_grid_tree,
    scoring='neg_root_mean_squared_error',
    verbose=1)

grid_tree.fit(features_train_final, target_train)

best_model_tree = grid_tree.best_estimator_
best_parametrs_tree = grid_tree.best_params_
best_result_tree = grid_tree.best_score_ * -1

Fitting 5 folds for each of 13 candidates, totalling 65 fits


In [41]:
print(f'Лучший результат RMSE модели: {best_result_tree}') 
print(f'Лучшие параметры модели: {best_parametrs_tree}') 

Лучший результат RMSE модели: 1947.7491302865765
Лучшие параметры модели: {'decisiontreeregressor__max_depth': 14}


In [42]:
%%time

predicted_test = best_model_tree.predict(features_train_final)

CPU times: user 256 ms, sys: 225 ms, total: 482 ms
Wall time: 486 ms


In [43]:
rmse_score_diff_methods.loc['DecisionTree', 'RMSE'] = best_result_tree
rmse_score_diff_methods.loc['DecisionTree', 'time_fit at s'] = grid_tree.cv_results_['mean_fit_time'].mean()
rmse_score_diff_methods.loc['DecisionTree', 'time_predict'] = '258 ms'

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
LGBMRegressor
</div>

In [44]:
params = {'learning_rate': [0.5, 0.1],
          'max_depth': range(1, 8),
          'n_estimators': range(50, 351, 50)}

fit_params = {'categorical_feature': cat_col}

model = lgb.LGBMRegressor(random_state=42)
grid = RandomizedSearchCV(model, 
                          param_distributions=params, 
                          scoring='neg_root_mean_squared_error', 
                          cv=5,
                          n_jobs=-1, 
                          random_state=42,
                          verbose=1)
grid.fit(features_train_lgb, target_train, **fit_params)

best_model_lgb = grid.best_estimator_
best_parametrs_lgb = grid.best_params_
best_result_lgb = grid.best_score_ * -1

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [45]:
print(f'Лучший результат RMSE модели: {best_result_lgb}') # лучший результат RMSE модели
print(f'Лучшие параметры модели: {best_parametrs_lgb}') # лучшие параметры модели

Лучший результат RMSE модели: 1637.8238174953613
Лучшие параметры модели: {'n_estimators': 350, 'max_depth': 5, 'learning_rate': 0.1}


In [46]:
%%time

# замеряем время предсказания
predicted_test = best_model_lgb.predict(features_train_lgb)

CPU times: user 7.32 s, sys: 0 ns, total: 7.32 s
Wall time: 7.33 s


In [47]:
rmse_score_diff_methods.loc['LightGBM', 'RMSE'] = best_result_lgb
rmse_score_diff_methods.loc['LightGBM', 'time_fit at s'] = grid.cv_results_['mean_fit_time'].mean() +
rmse_score_diff_methods.loc['LightGBM', 'time_predict'] = '8.12 s'

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
CatBoost
</div>

In [48]:
params = {'learning_rate': [0.5, 0.1],
          'max_depth': range(1, 8),
          'n_estimators': range(50, 351, 50)}

fit_params = {'cat_features': cat_col}

model = CatBoostRegressor(random_state=42)
grid = RandomizedSearchCV(model, 
                          param_distributions=params, 
                          scoring='neg_root_mean_squared_error', 
                          cv=5,
                          n_jobs=-1, 
                          random_state=42,
                          verbose=1)
grid.fit(features_train_lgb, target_train, **fit_params)

best_model_cat = grid.best_estimator_
best_parametrs_cat = grid.best_params_
best_result_cat = grid.best_score_ * -1

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0:	learn: 4325.3576296	total: 119ms	remaining: 41.6s
1:	learn: 4149.7726044	total: 181ms	remaining: 31.5s
2:	learn: 3998.7185932	total: 244ms	remaining: 28.2s
3:	learn: 3856.4234096	total: 291ms	remaining: 25.2s
4:	learn: 3733.8823223	total: 324ms	remaining: 22.4s
5:	learn: 3629.0002328	total: 385ms	remaining: 22s
6:	learn: 3521.5037260	total: 428ms	remaining: 21s
7:	learn: 3438.9594304	total: 476ms	remaining: 20.4s
8:	learn: 3344.3070157	total: 535ms	remaining: 20.3s
9:	learn: 3263.0286992	total: 589ms	remaining: 20s
10:	learn: 3193.8128103	total: 639ms	remaining: 19.7s
11:	learn: 3123.4023550	total: 673ms	remaining: 18.9s
12:	learn: 3067.1502514	total: 719ms	remaining: 18.6s
13:	learn: 3016.0530591	total: 772ms	remaining: 18.5s
14:	learn: 2966.6825238	total: 812ms	remaining: 18.1s
15:	learn: 2924.8070671	total: 867ms	remaining: 18.1s
16:	learn: 2878.8127289	total: 920ms	remaining: 18s
17:	learn: 2835.9552398	total: 968ms	re

In [49]:
print(f'Лучший результат RMSE модели: {best_result_cat}') # лучший результат RMSE модели
print(f'Лучшие параметры модели: {best_parametrs_cat}') # лучшие параметры модели

Лучший результат RMSE модели: 1657.5229407997522
Лучшие параметры модели: {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.5}


In [50]:
%%time

# замеряем время предсказания
predicted_test = best_model_cat.predict(features_train)

CPU times: user 699 ms, sys: 1.1 ms, total: 700 ms
Wall time: 701 ms


In [55]:
rmse_score_diff_methods.loc['CatBoost', 'RMSE'] = best_result_cat
rmse_score_diff_methods.loc['CatBoost', 'time_fit at s'] = grid.cv_results_['mean_fit_time'].mean()
rmse_score_diff_methods.loc['CatBoost', 'time_predict'] = '926 ms'

<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Модели обучены, предсказания сделаны, переходим к проверке метрики rmse
</div>

In [56]:
rmse_score_diff_methods

,RMSE,time_fit,time_predict,time_fit at s
Ridge,2641.927758,3.580244,119 ms,NaN
DecisionTree,1947.749130,3.215187,258 ms,NaN
LightGBM,1637.823817,11.051676,8.12 s,NaN
CatBoost,1657.522941,19.391378,926 ms,19.391378


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
По совокупности результутатов лучшая модель LGBMRegressor, CatBoostRegressor так же хороший показатель метрики, но время обученения слишком большое :с
</div>

In [53]:
%%time

model = lgb.LGBMRegressor(n_estimators=350, max_depth=5, learning_rate=0.1, random_state=42)
model.fit(features_train_lgb, target_train, categorical_feature=cat_col)

CPU times: user 28 s, sys: 50.5 ms, total: 28.1 s
Wall time: 28.2 s


LGBMRegressor(max_depth=5, n_estimators=350, random_state=42)

In [54]:
%%time

predicted_test = model.predict(features_test_lgb)

print(f'RMSE на тестовой выборке: {round(math.sqrt(mean_squared_error(target_test, predicted_test)), 2)}')

RMSE на тестовой выборке: 1609.75
CPU times: user 2.71 s, sys: 0 ns, total: 2.71 s
Wall time: 2.71 s


<div class="alert alert-info" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px "> 
Итоговые показатели на тестовой выборке у лучшей модели: метрика rmse 1610.1, время обучения 23.7s, время время предсказания 2.74s
</div>